In [2]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [3]:
try:
    __confing.execute_notebook('notebooks/data_extract.ipynb')
except:
    pass

In [4]:
import pandas as pd
import numpy as np

from src import value_filler as vs
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [49]:
n='a'
df=pd.read_csv(r'data/custom/w1_and_w2.csv')
target=f'p{n}hosp1y'
drop=[f'p{n}hspnit1y'] # si la wave3 falla quitar estas columnas , 'r3bpsft', 'r3bpref'

In [50]:
# Drop
df = df.drop(drop, axis=1)

df = df.dropna(subset=[target]) 

porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.4)]

In [59]:
# fill
y = df[target]
X = df.drop(target,axis=1)
X = fs.fast_fill_2(X)

In [60]:
# # feature selection
# feature_selected=['paoophos1y', 'pameals', 'pahigov', 'parifaany', 'pamweight', 'pahghtsft', 'pauppermobm', 'pahenum', 'pahrtatte', 'paoophosf1y', 'pabalrtsec', 'panagi8', 'parxhrtat', 'pawghtsft', 'paeffort', 'paoopmdf1y', 'paslfemp', 'parfaany', 'pahrtatlmt', 'pacagem', 'pabaltryu', 'panovisual', 'pahtcomp', 'padoctor1y', 'padoctim1y', 'parfcare', 'pacholst', 'pamedsa', 'pafimrc_m', 'parelgimp', 'paeat', 'pafallinj', 'paricany', 'padrinkbd', 'pampart', 'pawork', 'pagrossa', 'panagi10', 'pagpcarehr', 'palowermob', 'pambmi', 'pastoop', 'pabalrt', 'pasita', 'pabede', 'parxhibp', 'paarmsa', 'pamomliv', 'paoopdoc1y', 'pajlocc_m', 'pajcten', 'pamheight', 'pawhappy', 'paswell', 'pawalk1', 'padrinkcr', 'parxdiabi', 'papusha', 'palifein_m', 'paoutpt1y', 'pasleepr', 'pashlt', 'paimrc_m', 'parccarehr', 'padrinkn', 'paracaany', 'pagpcare', 'pariccaren', 'parccare', 'pahltc']
# X=X[feature_selected]

In [61]:
# feature selection
feature_selected=['paurina2y','pacholst','pabreast','paprost','papapsm','pamammog','pacage','padrinkcr','padrinkbd','padrinkb','paheight','paweight','pabmi','papaina','papainlv','papainfr','pafatigue','pawheeze','pabreath_m','paswell','pahearaid','pahearing','pasight','paglasses','palunglmt_m','pahrtatlmt','pastroklmt','paarthlmt','palowermoba','pauppermoba','pagrossaa','palgmusaa','pamobilaa','paclims','pasit','paarms','palift','pastoop','pachair','pameals','pashop','pamoney','pameds','pabedhlp','pabed','patoilt','paeat','pabath','pawalkr','padress','pahipcomp','pafall','pafallinj','pamhip','pahipe_m','parxarthr','parxstrok','parxhrtat','pacncrothr','pacncrmeds','pacncrradn','pacncrsurg','pacncrchem','parxdiab','parxdiabo','parxhibp','parechrtatt','parifaany','parfaany','parfcaren','pararcare','padresshlp', 'pawalkhlp', 'pabathehlp', 'paeathlp', 'pabedhlp', 'patoilethlp', 'pamealhlp', 'pashophlp', 'pamedhlp', 'pamoneyhlp','pahibpe','pagender','padiabe','pacancre','palunglmt_m','parxlung_m','pahrtatte','pastroke','paarthre','parifaany','parafaany','pahigov','pasmokev','pasmoken','paoangry','paosleep','paodngr','paodngr','paopace','paoplot','paoalchl','pawthh','paagey','pamomage','padadage','paprmem','parjudg','parorgnz']
X=X[feature_selected]

In [62]:
# split
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)
X_test_o, y_test_o = X_test, y_test

In [63]:
# resample 
X_train, y_train = dp.apply_resample(X_train, y_train,v=1.20)

In [64]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [65]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [66]:
best_params={
    'max_depth': 8, 
    'learning_rate': 0.01759366380037561, 
    'n_estimators': 870, 
    'subsample': 0.4151616264321625, 
    'colsample_bytree': 0.18567068433714898
}

k = 8
kf = KFold(n_splits=k)

features = X_train
labels = y_train

for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    model = XGBClassifier(**best_params)
    model.fit(X_train, y_train)

In [67]:
y_pred_proba = model.predict_proba(X_test_o)[:, 1]
auc = roc_auc_score(y_test_o, y_pred_proba)
print(f'AUC={auc}')
print(y_test_o.value_counts())

AUC=0.8384045216212074
pahosp1y
0.0    3279
1.0     985
Name: count, dtype: int64


In [68]:
def feature_importance(model, df:pd.DataFrame, target:str, n:int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(df.columns)
    feature_names_original.remove(target)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista

In [69]:
feature_selected = feature_importance(model, df, target, n=70)
print(feature_selected)

['pahltc', 'padressa', 'pawalkra', 'pashopa', 'paiwy', 'pamrct', 'pamealsa', 'pamstath', 'papusha', 'pawalkr', 'paagey', 'paiwm', 'pabath', 'pabeda', 'palifta', 'pasita', 'pachair', 'pabatha', 'pampart', 'paproxy', 'Unnamed: 0', 'paarmsa', 'patoilt', 'pawtresp', 'pabed', 'paclim1', 'pamstat', 'pashop', 'paadlfivem', 'pamoneya', 'pajoga', 'padress', 'paadlfive', 'pasit', 'paadlaa', 'paclims', 'paadla_m', 'paeata', 'pajog', 'paeat', 'palift', 'paclim1a', 'pastoopa', 'padimea', 'pameals', 'pamoney', 'pawalk1', 'pawalksa', 'pamedsa', 'pameds', 'pawalk1a', 'paadlam', 'paadla', 'pawalks', 'patoilta', 'paadltota_m', 'paadltotm_m', 'paadlfivea', 'pashlt', 'paiwf', 'paadltot_m', 'padime', 'pachaira', 'papush', 'paarms', 'paiwstat', 'paclimsa', 'pastoop']


In [70]:
best_params={
    'max_depth': 8, 
    'learning_rate': 0.01759366380037561, 
    'n_estimators': 870, 
    'subsample': 0.4151616264321625, 
    'colsample_bytree': 0.18567068433714898
}# con variables de gastos medicos 92%

In [71]:
best_params={
    'max_depth': 20,
    'learning_rate': 0.004329892739635385,
    'n_estimators': 797,
    'subsample': 0.9333420520310073,
    'colsample_bytree': 0.2422950171008133
}
# sin varaibels de gastos medicos 89%